# Power Spectrum Summary

**by Josh Dillon and Steven Murray**, last updated March 25, 2025

The purpose of this notebook is to pull together results from power spectra from single, redundantly-averaged baselines (typically cross-power spectra from interleaved sets of times) as produced by the [Single Baseline Filtering and Power Spectrum Estimation
notebook](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/single_baseline_postprocessing_and_pspec.ipynb). It is supposed to be roughly comparable to [a similar notebook from H1C](https://github.com/HERA-Team/H1C_IDR3_Power_Spectra/blob/main/SPOILERS/All_Epochs_Power_Spectra/H1C_IDR3_Power_Spectra.ipynb).

### [• Figure 1: Per-Baseline Signal Loss Corrections](#Figure-2:-Per-Baseline-Signal-Loss-Corrections)
### [• Figure 2: Per-Baseline, Time-Averaged High Delay Average SNR](#Figure-3:-Per-Baseline,-Time-Averaged-High-Delay-Average-SNR)
### [• Figure 3: Histograms of Time-Averaged High-Delay SNRs](#Figure-4:-Histograms-of-Time-Averaged-High-Delay-SNRs)
### [• Figure 4: Time-Averaged Cylindrical P(k)](#Figure-5:-Time-Averaged-Cylindrical-P(k))
### [• Figure 5: Time-Averaged Cylindrical SNR](#Figure-6:-Time-Averaged-Cylindrical-SNR)
### [• Figure 6: Spherically-Averaged $\Delta^2$](#Figure-7:-Spherically-Averaged-%24%5CDelta%5E2%24)
### [• Table 1: Power Spectra, Error Bars, and Upper Limits](#Table-1:-Power-Spectra,-Error-Bars,-and-Upper-Limits)

## Imports and Parameters

In [ ]:
import time
tstart = time.time()

In [ ]:
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import h5py
import hdf5plugin  # REQUIRED to have the compression plugins available
import numpy as np
import glob
import copy
import pandas as pd
import pickle
from hera_cal import io, utils
import hera_pspec as hp
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
from scipy import constants
from scipy.signal import windows
from IPython.core.display import display, HTML
%matplotlib inline

In [ ]:
# Data settings
TAVG_PSPEC_FILE: str = '/lustre/aoc/projects/hera/h6c-analysis/IDR2/lstbin-outputs/redavg-smoothcal-inpaint-500ns-lstcal/inpaint/single_baseline_files/baselines_merged.tavg.pspec.h5'

FRF_SIGNAL_LOSS_FILE: str = None

# Output Files
RESULTS_FOLDER: str = '/lustre/aoc/projects/hera/smurray/hera/h6c-idr2-pspec/v3.3'
CASENAME: str = "all_baselines_interleaved_IDR2.3_500ns_14band"

WINDOW_FUNCTION_FILEGLOB: str = "/lustre/aoc/projects/hera/kfchen/H6C/inpainting/window_function/exact_window-spw??-wf.npy"

# Analysis Options
BANDS_TO_USE: str = "1,2,3,5,6,9,10,13"
WEDGE_BUFFER_NS: float = 500.0
MAX_FRF_SIGNAL_LOSS: float = 0.1

KBINS_DK_MULTIPLIER: float = 4.0        # How wide the |k|-bins are, multiplied by natural delay resolution
KBINS_KSTART_MULTIPLIER: float = 0.625  # The first |k| bin center is at KBINS_DK_MULTIPLIER*dk*KBINS_KSTART_MULTIPLIER
KBINS_MAXK_HMPC: float = 2.5            #  The maximum k to include in the spherical averages

In [ ]:
# Some simple formatting of the inputs
BANDS_TO_USE = [int(band) for band in BANDS_TO_USE.split(",")] # 1 indexed

RESULTS_FOLDER = Path(RESULTS_FOLDER)

TAVG_PSPEC_FILE = Path(TAVG_PSPEC_FILE)
    
if FRF_SIGNAL_LOSS_FILE is None:
    FRF_SIGNAL_LOSS_FILE = TAVG_PSPEC_FILE.parent / TAVG_PSPEC_FILE.name.replace(".pspec.h5", ".frf_losses.pkl")
else:
    FRF_SIGNAL_LOSS_FILE = Path(FRF_SIGNAL_LOSS_FILE)
    
if WINDOW_FUNCTION_FILEGLOB is None:
    WINDOW_FUNCTION_FILEGLOB = f"{TAVG_PSPEC_FILE.parent}/zen.LST.baseline.*.sum.tavg.pspec.window.pkl"


## Load Power Spectra



In [ ]:
psc = hp.PSpecContainer(TAVG_PSPEC_FILE, mode='r', keep_open=False)
uvp = psc.get_pspec('stokespol', 'time_and_interleave_averaged')
    
with open(FRF_SIGNAL_LOSS_FILE, 'rb') as f:
    frf_losses = pickle.load(f)

Add window functions. Non-exact window function files are produced in the `compute_window_functions.ipynb` notebook. Exact window functions produced by Kai-Feng Chen.

In [ ]:
uvp.select(
    spws=[b - 1 for b in BANDS_TO_USE],
    polpairs=[('pI', 'pI'), ('pQ', 'pQ')]
)

frf_losses = {k: v[[b - 1 for b in BANDS_TO_USE]] for k, v in frf_losses.items()}

Apply loss-correction from non-redundancy

**TODO: move this to single-baseline notebook**

In [ ]:
# These are losses in percent, taken from Rajorshi.
non_redundancy_losses = [1.2, 2.2, 1.7, 1.8, 1.3, 1.3, 2.1, 3.1]

for spw, loss in enumerate(non_redundancy_losses):
    uvp.data_array[spw] *= 100/(100 - loss)

In [ ]:
# dict to map baseline pairs to baseline vectors
blp_to_blvec_dict = dict(zip(uvp.get_blpairs(), uvp.get_blpair_blvecs()))

In [ ]:
# get mean redshifts for each spw
zs = np.array([np.mean(1.420405751e9 / uvp.freq_array[uvp.spw_freq_array == spw] - 1) for spw in uvp.spw_array])

Define the spherical k-bins

In [ ]:
spherical_kbins = {}
for spw in uvp.spw_array:
    dk = KBINS_DK_MULTIPLIER * np.median(np.diff(uvp.get_kparas(spw)))
    spherical_kbins[spw] = np.arange(KBINS_KSTART_MULTIPLIER * dk, KBINS_MAXK_HMPC, dk)

## Examine Signal Loss Across Baselines

In [ ]:
def plot_baseline_signal_loss():
    # Show signal loss correction applied to each baseline as a function of spectral window, highlighting those with too much signal loss 
    fig, axes = plt.subplots(int(np.ceil(uvp.Nspws / 2)),2, figsize=(12, 12), sharex=True, sharey=True, gridspec_kw={'wspace': 0, 'hspace':0})
    
    for spw, ax in enumerate(axes.ravel()):
        if spw == len(uvp.spw_array):
            break
    
        bl_vecs = {}
        for key in uvp.get_all_keys():
            _spw, blp, pp = key
            if _spw != spw: 
                continue
            if pp[0] != 'pI':
                continue
            blv = blp_to_blvec_dict[blp]
            if blv[1] < 0:
                bl_vecs[blp] = -blv
            else:
                bl_vecs[blp] = blv
                    
        if spw % 2 == 0:
            ax.set_ylabel('NS Baseline Component (m)')
        if spw >= int(np.ceil(uvp.Nspws / 2)) - 2:
            ax.set_xlabel('EW Baseline Component (m)')
        
        blps = list(bl_vecs.keys())
        im = ax.scatter([bl_vecs[blp][0] for blp in blps], [bl_vecs[blp][1] for blp in blps], c=[frf_losses[blp][spw] for blp in blps], 
                        edgecolors=['r' if (frf_losses[blp][spw] > MAX_FRF_SIGNAL_LOSS) else 'none' for blp in blps], linewidths=.5,
                        s=20, cmap='viridis', vmin=0, vmax=min(2 * MAX_FRF_SIGNAL_LOSS, 1))
    
        ax.text(ax.get_xlim()[0] + 10, ax.get_ylim()[-1] - 10, f'Band {spw + 1}\nz = {zs[spw]:.1f}', ha='left', va='top',
                         bbox=dict(facecolor='w', edgecolor='black', alpha=.75, boxstyle='round', ls='-'))
        
    plt.tight_layout()
    plt.colorbar(im, ax=axes, pad=.02, aspect=40, extend='max', location='top', label=f'Signal Loss due to Fringe Rate and Cross Talk Filters')
    plt.savefig(os.path.join(RESULTS_FOLDER, 'per_baseline_signal_loss_all_bands.pdf'), bbox_inches='tight')

### Figure 1: Per-Baseline Signal Loss Corrections
Baselines outlined in red have too much signal loss and are excluded from the final power spectrum.

In [ ]:
plot_baseline_signal_loss()

In [ ]:
def plot_baseline_SNR():
    # Look for individual baselines with high SNR outside the wedge, a sign of a particular failure mode
    low_dly = 1000
    high_dly = 2000
    fig, axes = plt.subplots(int(np.ceil(uvp.Nspws / 2)),2, figsize=(12, 12), sharex=True, sharey=True, gridspec_kw={'wspace': 0, 'hspace':0})
    
    for spw, ax in enumerate(axes.ravel()):
        if spw == len(uvp.spw_array):
            break
        bl_vecs = {}
        pk_avgs = {}
        snr_avgs = {}
        dlys = uvp.get_dlys(spw) * 1e9
        dlys_to_use = (low_dly <= np.abs(dlys)) & (np.abs(dlys) <= high_dly)
        for key in uvp.get_all_keys():
            _spw, blp, pp = key
            if _spw != spw: 
                continue
            if pp[0] != 'pI':
                continue
            blv = blp_to_blvec_dict[blp]
            if blv[1] < 0:
                bl_vecs[blp] = -blv
            else:
                bl_vecs[blp] = blv
            pk_avgs[blp] = np.mean(uvp.get_data(key)[0, dlys_to_use].real)
            snr_avgs[blp] = pk_avgs[blp] / np.mean(uvp.get_stats('P_N', key)[0, dlys_to_use].real)
            
        if spw % 2 == 0:
            ax.set_ylabel('NS Baseline Component (m)')
        if spw >= int(np.ceil(uvp.Nspws / 2)) - 2:
            ax.set_xlabel('EW Baseline Component (m)')
        
        blps = list(bl_vecs.keys())
        
        im = ax.scatter([bl_vecs[blp][0] for blp in blps], [bl_vecs[blp][1] for blp in blps], c=[snr_avgs[blp] for blp in blps], 
                        edgecolors=['k' if (frf_losses[blp][spw] >= MAX_FRF_SIGNAL_LOSS) else 'none' for blp in blps], linewidths=.5,
                        s=20, cmap='bwr', vmin=-2, vmax=2)
    
        ax.text(ax.get_xlim()[0] + 10, ax.get_ylim()[-1] - 10, f'Band {spw + 1}\nz = {zs[spw]:.1f}', ha='left', va='top',
                         bbox=dict(facecolor='w', edgecolor='black', alpha=.75, boxstyle='round', ls='-'))
        
    plt.tight_layout()
    plt.colorbar(im, ax=axes, pad=.02, aspect=40, extend='both', location='top', label=f'Average $P(k) / P_N$ Between {low_dly} and {high_dly} ns')
    plt.savefig(os.path.join(RESULTS_FOLDER, 'per_baseline_SNR_all_bands.pdf'), bbox_inches='tight')

### Figure 2: Per-Baseline, Time-Averaged High Delay Average SNR
Baselines outlined in black have too much signal loss and are excluded from the final power spectrum.

In [ ]:
plot_baseline_SNR()

### Remove Baselines with High FRF Signal Loss

In [ ]:
uvp_spws = {}
for spw in uvp.spw_array:
    blps_to_use = [blp for blp in uvp.get_blpairs() if frf_losses[blp][spw] <= MAX_FRF_SIGNAL_LOSS]
    uvp_spws[spw] = uvp.select(spws=[spw], blpairs=blps_to_use, inplace=False)
del uvp

## Plot kperp vs |k| to check that each kpar line stays inside a bin

In [ ]:
fig, ax = plt.subplots(len(BANDS_TO_USE), 1, figsize=(12, 15),layout='constrained')

cnv = hp.conversions.Cosmo_Conversions()

for i, (spw, _uvp) in enumerate(uvp_spws.items()):
    for kk in np.sort(_uvp.get_kperps(0))[::15]:
        kpar = _uvp.get_kparas(0)
        kpar = kpar[kpar>0]
        kmag = np.sqrt(kpar**2 + kk**2)
        
        ax[i].scatter(kmag, np.ones_like(kmag)*kk, s=2, c='C0')    

    ax[i].set_xlim(-0.05, kmag.max()+0.05)
        
    b_to_kperp = lambda b: cnv.bl_to_kperp(zs[spw]) * b
    kperp_to_b = lambda k: k / cnv.bl_to_kperp(zs[spw])
    kpar_to_tau = lambda k: k / cnv.tau_to_kpara(zs[spw]) * 1e9
    tau_to_kpar = lambda tau: cnv.tau_to_kpara(zs[spw]) * tau / 1e9

    # Horizon
    kperp = np.linspace(0, _uvp.get_kperps(0).max(), 100)
    horizon_k = np.sqrt(tau_to_kpar(1e9 * kperp_to_b(kperp)/3e8 + 500)**2 + kperp**2)
    ax[i].plot(horizon_k, kperp, color='r')

    ax[i].set_ylim(0, _uvp.get_kperps(0).max())

    dk = spherical_kbins[spw][1] -spherical_kbins[spw][0]
    for sph in spherical_kbins[spw]:
        ax[i].axvline(sph - dk/2 , color='k')
        
    ax[i].axvline(spherical_kbins[spw][-1] + dk/2, color='k')

### Add Window Functions

In [ ]:
window_function_files = sorted(glob.glob(WINDOW_FUNCTION_FILEGLOB))

if len(window_function_files)== 1:
    # Old-style window functions. Neither time nor baseline dependent.
    with open(window_function_files[0], 'rb') as fl:
        window_functions = pickle.load(fl)
    uvp.window_function_array = {k: np.tile(window_functions[k], (uvp.Nbltpairs, 1, 1, 1)) for k in window_functions}
    uvp.exact_window_functions = False
    
else:
    # Exact window functions, 
    wf_folder = os.path.dirname(window_function_files[0])

    for spw, band in enumerate(BANDS_TO_USE):
        kperpbins = np.load(f"{wf_folder}/exact_window-spw{band-1:02d}-kperp.npy")
        kparabins = np.load(f"{wf_folder}/exact_window-spw{band-1:02d}-kpara.npy")
        wf_full = np.load(f"{wf_folder}/exact_window-spw{band-1:02d}-wf.npy")

        uvp_spws[spw].exact_windows = True
        uvp_spws[spw].window_function_array = {0: wf_full}
        uvp_spws[spw].window_function_kperp = {0: kperpbins}
        uvp_spws[spw].window_function_kpara = {0: kparabins}

## Examine Cylindrical Averages at Raw Resolution

In [ ]:
def plot_SNR_histograms(uvps: dict[int, hp.UVPSpec]):
    # show the distribution of time-averaged power spectra (before spherical averaging)
    low_dly = 1000
    high_dly = 2000
    nspws = len(uvps)
    fig, axes = plt.subplots(int(np.ceil(nspws / 2)), 2, figsize=(12, 12), sharex=True, sharey=True, gridspec_kw={'wspace': 0, 'hspace':0})
    
    for spw, uvp in uvps.items():
        ax = axes.flatten()[spw]

        to_hist = []
        dlys = uvp.get_dlys(0) * 1e9
        dlys_to_use = (low_dly <= np.abs(dlys)) & (np.abs(dlys) <= high_dly)
        for key in uvp.get_all_keys():
            _spw, blp, pp = key
            if pp[0] != 'pI':
                continue
            
            SNRs = uvp.get_data(key)[0, dlys_to_use] / uvp.get_stats('P_N', key)[0, dlys_to_use].real
            to_hist.extend(SNRs)
    
        bins = np.linspace(-10,10,101)        
        
        for func, c, zorder in zip([np.real, np.imag], ['C0', 'C1'], [2, 1]):
            ax.hist(func(to_hist), bins=bins, density=True, color=c, edgecolor='k', linewidth=.1, alpha=.5, zorder=zorder,
                    label=f'{"Re" if func == np.real else "Im"}[$P(k)$] / $P_N$ ({low_dly}—{high_dly} ns)')
        ax.set_yscale('log')
        ax.set_ylim([2e-5, 1e0])
        
        gauss = np.exp(-bins**2/2) / np.sqrt(2*np.pi)
        ax.plot(bins, gauss, 'k--', label='Gaussian Distribution')
    
        ax.text(ax.get_xlim()[0] + .6, ax.get_ylim()[-1] / 1.8, f'Band {spw + 1}\nz = {zs[spw]:.1f}', ha='left', va='top',
                bbox=dict(facecolor='w', edgecolor='black', alpha=.75, boxstyle='round', ls='-'))
        if spw % 2 == 0:
            ax.set_ylabel('Probability Density')
        if spw >= int(np.ceil(uvp.Nspws / 2)) - 2:
            ax.set_xlabel('SNR')
    
        if spw == 0:
            fig.legend(loc='upper center', ncol=3, fontsize=10)
    plt.tight_layout()
    fig.subplots_adjust(top=0.95)
    
    plt.savefig(os.path.join(RESULTS_FOLDER, 'SNR_histogram_all_bands.pdf'), bbox_inches='tight')

### Figure 4: Histograms of High-Delay SNRs

In [ ]:
plot_SNR_histograms(uvp_spws)

In [ ]:
def plot_cylindrical_Pk(uvps: dict[int, hp.UVPSpec], SNR: bool=False):
    nspws = len(uvps)
    fig, axes = plt.subplots(int(np.ceil(nspws / 2)),2, figsize=(12, 12), 
                             sharex=True, sharey=True, gridspec_kw={'wspace': 0, 'hspace':0}, constrained_layout=True)

    cnv = hp.conversions.Cosmo_Conversions()
    
    for spw, uvp in uvps.items():
        uvp_here = copy.deepcopy(uvp)
        ax = axes.flatten()[spw]
        # In each uvp, the actual spw is zero.
        _spw = 0

        kbins = spherical_kbins[spw].copy()
        kbins -= (kbins[1] - kbins[0])/2 # Get k-edges.
        
        b_to_kperp = lambda b: cnv.bl_to_kperp(zs[spw]) * b
        kperp_to_b = lambda k: k / cnv.bl_to_kperp(zs[spw])
        kpar_to_tau = lambda k: k / cnv.tau_to_kpara(zs[spw]) * 1e9
        tau_to_kpar = lambda tau: cnv.tau_to_kpara(zs[spw]) * tau / 1e9
            
        if SNR:
            uvp_here.data_array[_spw] /= uvp_here.stats_array['P_N'][_spw].real
            extra_kwargs = dict(cmap='bwr', vmin=-2, vmax=2)
        else:
            extra_kwargs = dict(cmap='turbo', error_weights='P_N')
        
        hp.plot.delay_wedge(uvp_here, _spw, ('pI', 'pI'), rotate=True, ax=ax, fold=not uvp.folded, log10=False, 
                            component='real',  horizon_lines=True, **extra_kwargs)

        # Plot the wedge + buffer
        b = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 100)
        ylim = ax.get_ylim()  # re-set afterwards to have nice ylims
        ax.plot(b, b * 1e9 / constants.c + WEDGE_BUFFER_NS, color='k', ls='--')

        # Plot final spherical k-bins
        kperp = b_to_kperp(b)
        for kedge in kbins:
            bintau = kpar_to_tau(np.sqrt(kedge**2 - kperp**2))
            ax.plot(b, bintau, color='gray', lw=1)
        
        cax = ax.collections[0]
        if not SNR:
            cax.set_norm(matplotlib.colors.LogNorm(vmin=1e2, vmax=1e14))

        if spw % 2 == 0:
            ax.set_ylabel('$\\tau$ (ns)', fontsize=10)
        else:
            ax.set_ylabel('')
        if uvp.Nspws - spw <= 2:
            ax.set_xlabel('$|\\vec{b}|$ (m)', fontsize=10)
        else:
            ax.set_xlabel('')

        if spw < 2:
            # Mark kperp and kpar
            twinx = ax.secondary_xaxis('top', functions=(b_to_kperp, kperp_to_b))
            twinx.set_xlabel(r"$k_\perp \ \ [h/{\rm Mpc}]$")
            
        if spw %2 == 1:
            twiny = ax.secondary_yaxis('right', functions=(tau_to_kpar, kpar_to_tau))
            twiny.set_ylabel(r"$k_{||} \ \ [h/{\rm Mpc}]$")
            
        ax.set_ylim(ylim)
        ax.text(ax.get_xlim()[0] + 7, ax.get_ylim()[-1] - 200, f'Band {spw + 1}\nz = {zs[spw]:.1f}', ha='left', va='top',
                     bbox=dict(facecolor='w', edgecolor='black', alpha=.75, boxstyle='round', ls='-'))
        
        ax.tick_params(axis='both', direction='in')
    
    cbar = fig.colorbar(cax, ax=axes, pad=.02, aspect=40, extend='both', location='top',
                        label=(r"$\text{Re}[P(k_\parallel, k_\perp)]\text{ }/\text{ }P_N(k_\parallel, k_\perp)$" if SNR
                               else r"$\text{Re}[P(k_\parallel, k_\perp)]\ [{\rm mK}^2\ h^{-3}\ {\rm Mpc}^3]$"))
    
    #plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_FOLDER, (f'cylindrical_{"SNR" if SNR else "Pk"}_all_bands.pdf')), bbox_inches='tight')

### Figure 5: Cylindrical P(k)

In [ ]:
plot_cylindrical_Pk(uvp_spws)

### Figure 6: Cylindrical SNR

In [ ]:
plot_cylindrical_Pk(uvp_spws, SNR=True)

## Examine Cylindrical Spectra after Delay-Downsampling

### Average over bins of delay

First, we need to add a covariance matrix to each UVPSpec object. In the future, this should be properly computed elsewhere, but for now, we use an approximation that is very quick to compute:

In [ ]:
for _uvp in uvp_spws.values():
    _uvp.add_approximate_covariance()
    

Now, average within delay bins, using a four-bin kernel whose outer bins are only 2% of the total weight. This results in covariance between neighbouring (downsampled) delay-bins that is less than ~1%.

In [ ]:
uvp_downsampled = {
    spw:  hp.grouping.average_in_delay_bins(
        uvp = _uvp, 
        kernel = np.array([0.02, 0.48, 0.48, 0.02]), 
    ) for spw, _uvp in uvp_spws.items()
}

### Figure 7: Time-Averaged Cylindrical P(k) downsampled

In [ ]:
plot_cylindrical_Pk(uvp_downsampled, SNR=False)

### Figure 8: Cylindrical SNR Downsampled

In [ ]:
plot_cylindrical_Pk(uvp_downsampled, SNR=True)

In [ ]:
def get_cylindrical_uvp(
    uvp, spw, pol, error_weights=None, fold=True, delay=True,
    red_tol=1.0, deltasq=False, 
):
    """Do necessary preparation to get a cylindrically-averaged UVP.
    
    Most of this function was ripped form plots.delay_wedge.
    """
    
    # type checking
    uvp = copy.deepcopy(uvp)
    assert isinstance(spw, (int, np.integer))
    assert isinstance(pol, (int, np.integer, tuple))
    
    # check pspec units for little h
    little_h = 'h^-3' in uvp.norm_units

    # Average across redundant groups and time
    # this also ensures blpairs are ordered from short_bl --> long_bl
    blp_grps, lens, angs, tags = hp.utils.get_blvec_reds(
        uvp, bl_error_tol=red_tol, match_bl_lens=True
    )
    uvp.average_spectra(blpair_groups=blp_grps, time_avg=True, error_weights=error_weights, inplace=True, error_field=[fld for fld in uvp.stats_array])
    
    # Convert to DeltaSq
    if deltasq:
        uvp.convert_to_deltasq(inplace=True)

    # Fold array
    if fold:
        uvp.fold_spectra()

    return uvp

def save_cylindrical_ps(uvp, spw):
    # This should take a pre-cylindricalised uvp
    _spw, _, pol = uvp.get_all_keys()[0]
    
    # get blpairs and order by len and enforce bl len ordering anyways
    blpairs, blpair_seps = uvp.get_blpairs(), uvp.get_blpair_seps()
    osort = np.argsort(blpair_seps)
    blpairs, blpair_seps = [blpairs[oi] for oi in osort], blpair_seps[osort]
    indices = np.squeeze([uvp.blpair_to_indices(blp) for blp in blpairs])
    pidx = uvp.polpair_to_indices(pol)[0]

    # get data with shape (Nblpairs, Ndlys)
    data = uvp.data_array[0][indices, :, pidx].real
    pn = uvp.stats_array['P_N'][0][indices, :, pidx]
    wf = uvp.window_function_array[0][indices, ..., pidx]
    kperp = uvp.get_kperps(0)[osort]
    kpara = uvp.get_kparas(0)
    cosmo = hp.conversions.Cosmo_Conversions()
    z = np.mean(cosmo.f2z(uvp.freq_array))
    
    np.savez(
        RESULTS_FOLDER / f"{CASENAME}.deltasq.2d.spw{spw+1:02}.npz", 
        deltasq = data, 
        errorbars_onesigma=pn, 
        window_function=wf, 
        window_function_kperp = uvp.window_function_kperp[0][:, pidx],
        window_function_kpara = uvp.window_function_kpara[0][:, pidx],
        kperp_h_on_mpc = kperp, 
        kpara_h_on_mpc=kpara, 
        z=z, 
        channel_freqs_hz=uvp.freq_array
    )

In [ ]:
cylindrical_uvps = {
    spw: get_cylindrical_uvp(_uvp, spw=0, pol=('pI', 'pI'), deltasq=True)
    for spw, _uvp in uvp_downsampled.items()
}

## Spherical Averaging

In [ ]:
sph_avgs = []
for spw, _uvp in uvp_downsampled.items():
    # make a copy of the time-averaged UVPspec object, where we can by-hand remove the baseline dependence of the time/lst arrays
    _uvp = copy.deepcopy(_uvp)
    for time_array in [
        _uvp.time_avg_array, _uvp.time_1_array, _uvp.time_2_array, 
        _uvp.lst_avg_array, _uvp.lst_1_array, _uvp.lst_2_array
    ]:
        time_array[:] = np.median(time_array)
    # no-op? take it out and see if it errors
    _uvp.average_spectra(time_avg=True, error_weights='P_N', error_field=['P_SN'], inplace=True)
    _uvp.set_stats_slice('P_N', 1e9 / constants.c, WEDGE_BUFFER_NS, above=False, val=np.inf)
    
    # Perform spherical averaging
    dk = spherical_kbins[spw][1] - spherical_kbins[spw][0]
    sph_avgs.append(hp.grouping.spherical_average(_uvp, spherical_kbins[spw], dk, error_weights='P_N'))
    
delta_sqs = [sph_avg.convert_to_deltasq(inplace=False) for sph_avg in sph_avgs]

In [ ]:
def plot_deltasq_limits(show_scale: bool = True):
    fig, axes = plt.subplots(
        2, int(np.ceil(len(uvp_spws) / 2)), 
        figsize=(12, 8), sharey='row', sharex=True, gridspec_kw={'wspace': 0, 'hspace': 0}
    )
    
    # Adjust layout to make room for the legend
    plt.subplots_adjust(top=0.95)
    
    # Loop over each subplot
    for spw, ax in enumerate(axes.ravel()):
    
        if spw >= len(uvp_spws) / 2:
            ax.set_xlabel('$k$ ($h$Mpc$^{-1}$)')
        if spw % int(np.ceil(len(uvp_spws) / 2)) == 0:
            ax.set_ylabel('$\\Delta^2$ (mK$^2$)')
        
        if spw == len(sph_avgs):
            break
    
        # get Delta^2 and error bars
        z = zs[spw]
        k = delta_sqs[spw].get_kparas(0)
        key = delta_sqs[spw].get_all_keys()[0]
        Dsq = np.squeeze(delta_sqs[spw].get_data(key)).real
        Dsq[Dsq < 0] = 0
        pn_error = np.squeeze(delta_sqs[spw].get_stats('P_N', key))
        psn_error = np.squeeze(delta_sqs[spw].get_stats('P_SN', key))
        pn_error = np.where(pn_error > 1e20, np.inf, pn_error)
        psn_error = np.where(psn_error > 1e20, np.inf, psn_error)
        
        # Plot with error bars and lines
        ax.errorbar(k, Dsq, yerr=(2 * psn_error), marker='o', ms=6, ls='', c='deeppink', label='1$\\sigma$ Noise Level')
        ax.plot(k, pn_error, c='k', ls='--', lw=3, label='PRELIMINARY Power Spectrum with 2$\\sigma$ Error Bars')
        ax.set_yscale('log')
        if spw >= len(uvp_spws) / 2:
            ax.set_ylim([1, 2e6])
        else:
            ax.set_ylim([1, 1e9])
        ax.grid()
        ax.text(ax.get_xlim()[-1] - .1, ax.get_ylim()[0]*3, f'Band {spw+1}\nz = {z:.1f}', ha='right', va='bottom',
                bbox=dict(facecolor='w', edgecolor='black', alpha=.75, boxstyle='round', ls='-'))
    
        # Add Phase I's two best limits at the most-comparable redshift
        if zs[np.argmin(np.abs(zs - 7.9))] == z:
            ax.errorbar([.34], [44], yerr=[2 * 206], marker='*', ms=10, lw=2, ls='', c='g', label='HERA Phase I Best Limits at Similar z')
            # Create a custom legend with labels
            handles, labels = axes.ravel()[spw].get_legend_handles_labels()
            fig.legend(handles, [labels[1], labels[0], labels[2]] , loc='upper center', ncol=3, fontsize=10)
        if zs[np.argmin(np.abs(zs - 10.4))] == z:
            ax.errorbar([.36], [0], yerr=[2 * 1748], marker='*', ms=10, lw=2, ls='', c='g')
    
        if not show_scale:
            ax.yaxis.set_ticklabels([])

### Figure 7: Spherically-Averaged $\Delta^2$

In [ ]:
plot_deltasq_limits(show_scale=False)

In [ ]:
def print_upper_limits_table():
    for spw in range(len(uvp_spws)):
        # get Delta^2 and error bars
        z = zs[spw]
        k = delta_sqs[spw].get_kparas(0)
        key = delta_sqs[spw].get_all_keys()[0]
        deltasq = np.squeeze(delta_sqs[spw].get_data(key)).real
        deltasq_err = np.squeeze(delta_sqs[spw].get_stats('P_SN', key)).real # TODO: this isn't quite what we did in H1C
        deltasq_ul = np.array([dsq if dsq > 0 else 0 for dsq in deltasq]) + 2 * deltasq_err
        to_use = (deltasq_err > 0) & (deltasq_err < 1e20)
        if len(deltasq_ul[to_use]) == 0:
            continue
    
        table = {'k': k[to_use],
                 r'$\Delta^{2}(k)$ (mK$^2$)': deltasq[to_use],
                 r'$1\sigma$ (mK$^2$)': deltasq_err[to_use],
                 r'$\Delta^{2}_{UL}$ (mK$^2$)': deltasq_ul[to_use]}
        df = pd.DataFrame(table)
    
        def css_border(x):
            return ["border-left: 1px solid black" if (i%3==1) else "border: 0px" for i, col in enumerate(x)]
        def scientific_html_formatter(x):
            formatted = "{:.2e}".format(x)  # Convert to scientific format like 8.58e+05
            base, exponent = formatted.split("e")  # Split into base and exponent
            exponent = int(exponent)  # Convert exponent to an integer
            return f"{base} × 10<sup>{exponent}</sup>"  # Format with HTML superscript
    
        display(HTML(f'<h3>Band {spw+1} (z = {zs[spw]:.1f}):</h3>'))
        
        to_display = df.style.format({'k': "{:,.2f}"} | {col: scientific_html_formatter for col in df.columns[1:]}) \
                              .apply(css_border, axis=1) \
                              .set_properties(width='100px') \
                              .hide(axis='index')
        
        display(to_display)

### Table 1: Power Spectra, Error Bars, and Upper Limits

In [ ]:
print_upper_limits_table()

## Export Limits 

In [ ]:
# results are saved to h5 files per-band and per-field
for spw, delta_sq in enumerate(delta_sqs):
    outfilename = RESULTS_FOLDER / f'{CASENAME}.deltasq.1d.spw{spw+1:02}.h5'
    delta_sq.write_hdf5(outfilename, overwrite=True)  

In [ ]:
for spw, delta_sq in cylindrical_uvps.items():
    outfilename = RESULTS_FOLDER / f'{CASENAME}.deltasq.2d.spw{spw+1:02}.h5'
    delta_sq.write_hdf5(outfilename, overwrite=True)    

In [ ]:
# Export 1D Power in npz format
for spw, delta_sq in enumerate(delta_sqs):
    np.savez(
        f"{RESULTS_FOLDER}/{CASENAME}.deltasq.1d.spw{spw+1:02}.npz", 
        deltasq_mk2 = delta_sq.data_array[0][0,:, 0].real,
        k_h_on_mpc = delta_sq.get_kparas(0),
        onesigma_error = delta_sq.stats_array['P_N'][0][0,:,0],
        window_function = delta_sq.window_function_array[0][0, :, :, 0],
        z=np.mean(hp.conversions.Cosmo_Conversions().f2z(delta_sq.freq_array)),
        channel_freqs_hz = delta_sq.freq_array
    )   

In [ ]:
# Export 2D power in npz format
for spw, _uvp in cylindrical_uvps.items():
    save_cylindrical_ps(_uvp, spw)

## Metadata

In [ ]:
for repo in ['numpy', 'scipy', 'astropy', 'hera_cal', 'hera_qm', 'pandas',
             'hera_filters', 'hera_pspec', 'hera_notebook_templates', 'pyuvdata']:
    exec(f'from {repo} import __version__')
    print(f'{repo}: {__version__}')

In [ ]:
print(f'Finished execution in {(time.time() - tstart) / 60:.2f} minutes.')